# LGBM - New Feature + Baseline Rolling Window

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from utils import *
import sys
import xgboost as xgb
import lightgbm as lgb
from datetime import datetime

NUMBER_OF_LAGS = 4

sys.path.append("../../main/datasets/")
!ls  ../../main/datasets/

1.0v.zip


<hr>

## Defining metrics

Baseline_score function

In [2]:
def baseline_score(prediction, target, simulatedPrice):
    prediction = prediction.astype(int)

    return np.sum((prediction - np.maximum(prediction - target, 0) * 1.6)  * simulatedPrice)

Evaluation Metric

In [3]:
def feval(prediction, dtrain):
    
    prediction = prediction.astype(int)
    target = dtrain.get_label()

    simulatedPrice = dtrain.get_weight()
    
    return 'feval', np.sum((prediction - np.maximum(prediction - target, 0) * 1.6)  * simulatedPrice), True

<hr>

## Building our dataset
This notebook makes this step cleaner than the previous versions. So It'll be tidier and shorter than before!

In [4]:
infos, items, orders = read_data("../../main/datasets/")
print("Sanity checks...", infos.shape, items.shape, orders.shape)

Sanity checks... (10463, 3) (10463, 8) (2181955, 5)


In [5]:
# Changing our time signatures
process_time(orders)

In [6]:
df = dataset_builder(orders, items)

<hr>

## Feature building

**Adding 'is_new'**

In [7]:
# This cell adds a feature responsible for indicating if in the current week
# a given item has its first appearance.
orders_sorted_by_week = orders.sort_values('group_backwards', ascending=False)
weeks_grouped_by_items = orders_sorted_by_week.groupby('itemID', as_index=False)
items_first_appearance = weeks_grouped_by_items.first()[['itemID', 'group_backwards']]
items_first_appearance.rename(columns={'group_backwards':'first_appearance'}, inplace=True)
df['is_new'] = 0
df = pd.merge(df, items_first_appearance, left_on=['itemID'], right_on=['itemID'], how='left', validate='m:1')
df.loc[df['first_appearance'] == df['group_backwards'], 'is_new'] = 1
df['not_selling_yet'] = 0
df.loc[df['first_appearance'] < df['group_backwards'], 'not_selling_yet'] = 2
df.drop(columns=['first_appearance'], inplace=True)

**Cumulative sale by category**

In [8]:
# percentage_accum_cat_3 feature...
df = cumulative_sale_by_category(df, category='category3')

**Time Encoding**

In [9]:
# Encoding our weeks as a series of sines and cosines...
# This function will consider our period as a semester in a year,
# so we can try other types of time encoding later!
df = time_encoder(df, 'group_backwards', 26)

**Lags and diffs**

In [10]:
# This cell lags and diffs our feature 'orderSum'
shifting = df.copy()

for i in range(1, NUMBER_OF_LAGS + 1):
    # Carrying the data of weeks t-1
    shifting[f'orderSum_{i}'] = shifting.groupby('itemID')['orderSum'].shift(i)

    
    # Getting the difference of the orders and promotions between weeks t-1 and t-2...
    shifting[f'orderSum_diff_{i}'] = shifting.groupby('itemID')[f'orderSum_{i}'].diff()
    

**Rolling window "orderSum"**

In [11]:
%%time
# This cell creates rolling-window features based on 'orderSum' in our dataset!
item_group = shifting.groupby(["itemID", "group_backwards"]).agg({'orderSum':'sum'})

# We'll .shift(-1) because it sorts our "group_backwards", 
# so doing .shift(1) would cause a HUGE dataleak.
aux_shifting = item_group.groupby('itemID')[['orderSum']].shift(-1)

aux_shifting.sort_values(['itemID', 'group_backwards'], ascending=[True, False], inplace=True)

for i in range(3):
    rolled_window = aux_shifting.groupby(['itemID'], as_index=False)[['orderSum']].rolling(2 ** i).mean()
    rolled_window.rename(columns={'orderSum':f"orderSum_mean_rolled_{i}"}, inplace=True)
    shifting = pd.merge(shifting, rolled_window, left_on=['itemID', 'group_backwards'], right_on=['itemID', 'group_backwards'])

CPU times: user 39.3 s, sys: 358 ms, total: 39.6 s
Wall time: 39.9 s


In [12]:
# LGBM Says on docs that it automatically handles zero values as NaN,
# so we'll keep this standard...
shifting.fillna(0, inplace=True)

<hr>

## Maximum error
The maximum error we could get in this dataset would be just guessing the mean of our sales from weeks 1 to 12, and that's what the cell below is computing.

In [13]:
worst_possible_prediction = shifting.loc[shifting.group_backwards < 13]['orderSum'].mean()
prediction = np.full(shifting.loc[shifting.group_backwards == 13]['orderSum'].shape, worst_possible_prediction) # Array filled with the mean...
target = shifting.loc[shifting.group_backwards == 13]['orderSum']
print("Guessing the mean of 'orderSum' for all items in target", mse(target, prediction) ** 0.5)

Guessing the mean of 'orderSum' for all items in target 90.29706562119341


<hr>

## Dataset Splitting (Train until week 3 / Val. week 2/ Test week 1)
All my experiments will use weeks 13 to 3 as a train set, week 2 as our validation set and week 1 as a test set.

In [14]:
train = shifting.loc[shifting.group_backwards >= 3]
val = shifting.loc[shifting.group_backwards == 2]
test = shifting.loc[shifting.group_backwards == 1]

weights = infos.set_index('itemID')['simulationPrice'].to_dict()

w_train = train['itemID'].map(weights)
w_val = val['itemID'].map(weights)

In [15]:
# I recommend to the other members of the team keeping the
# datatypes of our datasets as Pandas DataFrames instead of Numpy,
# since It will easier to use Boosting Analysis frameworks
y_train = train['orderSum']
y_val = val['orderSum']
X_train = train.drop(columns=["orderSum"])
X_val = val.drop(columns=["orderSum"])

In [18]:
params = {
#           "objective" : "poisson",
          "objective" : "l1",
          "metric" :"rmse",
          "learning_rate" : 0.5,
          'verbosity': 1,
          'max_depth': 7,
          'num_leaves': 15,
          "min_data_in_leaf":3500,
         }

lgbtrain = lgb.Dataset(X_train, label = y_train, weight=w_train, categorical_feature=[6, 7, 8, 10, 11])
lgbvalid = lgb.Dataset(X_val, label = y_val, weight=w_val, categorical_feature=[6, 7, 8, 10, 11])

num_round = 1000
model = lgb.train(params,
                  lgbtrain,
                  num_round,
                  valid_sets = [lgbtrain, lgbvalid], 
                  verbose_eval=5,
                  early_stopping_rounds=5,
#                   fobj=objective,
                  feval=feval,
                 )

Training until validation scores don't improve for 5 rounds
[5]	training's rmse: 39.6219	training's feval: 4.1391e+06	valid_1's rmse: 44.6191	valid_1's feval: 268870
[10]	training's rmse: 38.9372	training's feval: 6.19685e+06	valid_1's rmse: 43.951	valid_1's feval: 439156
[15]	training's rmse: 38.946	training's feval: 6.21067e+06	valid_1's rmse: 43.9514	valid_1's feval: 441264
[20]	training's rmse: 38.941	training's feval: 6.24863e+06	valid_1's rmse: 43.9448	valid_1's feval: 455556
Early stopping, best iteration is:
[17]	training's rmse: 38.941	training's feval: 6.24863e+06	valid_1's rmse: 43.9448	valid_1's feval: 455556


In [17]:
shifting.columns

Index(['group_backwards', 'itemID', 'orderSum', 'brand', 'manufacturer',
       'customerRating', 'category1', 'category2', 'category3',
       'recommendedRetailPrice', 'is_new', 'not_selling_yet',
       'percentage_accum_category3', 'group_backwards_sin',
       'group_backwards_cos', 'orderSum_1', 'orderSum_diff_1', 'orderSum_2',
       'orderSum_diff_2', 'orderSum_3', 'orderSum_diff_3', 'orderSum_4',
       'orderSum_diff_4', 'orderSum_mean_rolled_0', 'orderSum_mean_rolled_1',
       'orderSum_mean_rolled_2'],
      dtype='object')

<hr>

## Dataset Splitting (Train until week 2 and test with week 1)
All my experiments will use weeks 13 to 2 as a train set and week 1 as test

In [17]:
train = shifting.loc[shifting.group_backwards >= 2]
val = shifting.loc[shifting.group_backwards == 1]
test = shifting.loc[shifting.group_backwards == 1]

weights = infos.set_index('itemID')['simulationPrice'].to_dict()

w_train = train['itemID'].map(weights)
w_val = val['itemID'].map(weights)

In [18]:
# I recommend to the other members of the team keeping the
# datatypes of our datasets as Pandas DataFrames instead of Numpy,
# since It will easier to use Boosting Analysis frameworks
y_train = train['orderSum']
y_val = val['orderSum']
X_train = train.drop(columns=["orderSum"])
X_val = val.drop(columns=["orderSum"])

In [19]:
params = {
#           "objective" : "poisson",
          "objective" : "l1",
          "metric" :"rmse",
          "learning_rate" : 0.5,
          'verbosity': 1,
          'max_depth': 6,
          'num_leaves': 32,
          "min_data_in_leaf":3000,
         }

lgbtrain = lgb.Dataset(X_train, label = y_train, weight=w_train, categorical_feature=[6,7,8,10])
lgbvalid = lgb.Dataset(X_val, label = y_val, weight=w_val, categorical_feature=[6,7,8,10])

num_round = 1000
model = lgb.train(params,
                  lgbtrain,
                  num_round,
                  valid_sets = [lgbtrain, lgbvalid], 
                  verbose_eval=5,
                  early_stopping_rounds=5,
#                   fobj=objective,
                  feval=feval,
                  
                 )

Training until validation scores don't improve for 5 rounds
[5]	training's rmse: 39.6521	training's feval: 6.13093e+06	valid_1's rmse: 43.0678	valid_1's feval: 439971
[10]	training's rmse: 39.653	training's feval: 6.15302e+06	valid_1's rmse: 43.0656	valid_1's feval: 440005
Early stopping, best iteration is:
[6]	training's rmse: 39.6518	training's feval: 6.1569e+06	valid_1's rmse: 43.0655	valid_1's feval: 440114


<hr>

### Utilities

**Predicting at test time**

In [21]:
y_test = test['orderSum']
X_test = test.drop(columns=["orderSum"])
final_predictions = model.predict(X_test)

In [22]:
final_predictions

array([0.0625, 0.4375, 0.    , ..., 0.    , 0.    , 0.    ])

In [23]:
final_predictions[final_predictions < 0] = 0

In [24]:
final_predictions

array([0.0625, 0.4375, 0.    , ..., 0.    , 0.    , 0.    ])

In [25]:
baseline_score(final_predictions, y_test.values, infos['simulationPrice'])

440114.07999999996

# Post Processing (based on Bruno's 1.8 Notebook)

In [26]:
orders.sort_values("group_backwards", ascending=False).groupby(["itemID"])["group_backwards"].first()

itemID
1        12
2         9
3        13
4        12
5        13
         ..
10450     2
10459     4
10460     6
10462     5
10463     3
Name: group_backwards, Length: 9840, dtype: int64

In [27]:
# Weekpair is negative, so this works:
first_fortnight_item = orders.sort_values("group_backwards",
                                     ascending=False)\
                          .groupby(["itemID"])["group_backwards"].first()
first_fortnight_item

itemID
1        12
2         9
3        13
4        12
5        13
         ..
10450     2
10459     4
10460     6
10462     5
10463     3
Name: group_backwards, Length: 9840, dtype: int64

In [28]:
new_items_value = 10  # The mode seems too high, so go with 10...
idx = X_test["itemID"].isin(first_fortnight_item[first_fortnight_item == 1].index)
final_predictions[idx] = new_items_value

**Baseline calculation**

In [29]:
baseline_score(final_predictions, y_test.values, infos['simulationPrice'])

52354.37